In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
import seaborn as sns  
sns.set()
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings('ignore')
%config InLineBackend.figure_format = 'retina'

In [3]:
data_train = pd.read_csv('../input/av-healthcare-analytics-ii/healthcare/train_data.csv')
data_test = pd.read_csv('../input/av-healthcare-analytics-ii/healthcare/test_data.csv')

In [4]:
data_train.head()

In [5]:
data_test.shape

In [6]:
la = LabelEncoder()
data_train['Stay'] = la.fit_transform(data_train['Stay'])
Y = data_train.loc[:, ['Stay']]

data_train.drop(['Stay'], axis = 1, inplace = True)
data_test['Flag'] = 1
data_train['Flag'] = 0

In [7]:
Y

In [8]:
data = pd.concat((data_train,data_test))

In [9]:
data.shape

In [10]:
data.head()

In [11]:
data['City_Code_Patient'] = data['City_Code_Patient'].fillna(-1)
data['Bed Grade'] = data['Bed Grade'].fillna(-1)

In [12]:
co = data.corr()
sns.heatmap(co)

In [13]:
data.isnull().sum()

In [14]:
data.dtypes


In [15]:
data['Type of Admission'].value_counts(), data['Severity of Illness'].value_counts()

In [16]:
def encode_cat_cols(data, cat_cols):
    le_dic = {}
    for col in cat_cols:
        le = LabelEncoder()
        le.fit(data[col].unique().tolist())
        data[col] = le.transform(data[[col]])
        le_dic[col] = le
         
    data['Type of Admission'] = data['Type of Admission'].map(
        {
            'Urgent': 0,
            'Emergency':1,
            'Trauma': 2
        }
    )
    data['Severity of Illness'] = data['Severity of Illness'].map(
        {
            'Minor':0,
            'Moderate':1,
            'Extreme':2
        }
    )
    return data, le_dic

In [17]:
cat_cols = [x for x in data.select_dtypes(['object']).columns]
cat_cols.remove('Severity of Illness')
cat_cols.remove('Type of Admission')
cat_cols

In [18]:
data, le_dic =  encode_cat_cols(data, cat_cols)

In [19]:
le_dic

In [20]:
data.head()


In [21]:
data['Severity of Illness'].value_counts()

In [22]:
data['mean_Admission_Deposit_per_illness']=data.groupby(['Severity of Illness'])['Admission_Deposit'].transform('mean')
data['mean_Admission_Deposit_per_admission']=data.groupby(['Type of Admission'])['Admission_Deposit'].transform('mean')
data['mean_Admission_Deposit_per_bed']=data.groupby(['Bed Grade'])['Admission_Deposit'].transform('mean')
data['mean_Admission_Deposit_per_department']=data.groupby(['Department'])['Admission_Deposit'].transform('mean')
data['mean_Admission_Deposit_per_hosp']=data.groupby(['Hospital_code'])['Admission_Deposit'].transform('mean')

data['sum_Admission_Deposit_per_illness']=data.groupby(['Severity of Illness'])['Admission_Deposit'].transform('sum')
data['sum_Admission_Deposit_per_admission']=data.groupby(['Type of Admission'])['Admission_Deposit'].transform('sum')
data['sum_Admission_Deposit_per_bed']=data.groupby(['Bed Grade'])['Admission_Deposit'].transform('sum')
data['sum_Admission_Deposit_per_department']=data.groupby(['Department'])['Admission_Deposit'].transform('sum')
data['sum_Admission_Deposit_per_hosp']=data.groupby(['Hospital_code'])['Admission_Deposit'].transform('sum')

data['max_Admission_Deposit_per_illness']=data.groupby(['Severity of Illness'])['Admission_Deposit'].transform('max')
data['max_Admission_Deposit_per_admission']=data.groupby(['Type of Admission'])['Admission_Deposit'].transform('max')
data['max_Admission_Deposit_per_bed']=data.groupby(['Bed Grade'])['Admission_Deposit'].transform('max')
data['max_Admission_Deposit_per_department']=data.groupby(['Department'])['Admission_Deposit'].transform('max')
data['max_Admission_Deposit_per_hosp']=data.groupby(['Hospital_code'])['Admission_Deposit'].transform('max')

data['min_Admission_Deposit_per_illness']=data.groupby(['Severity of Illness'])['Admission_Deposit'].transform('min')
data['min_Admission_Deposit_per_admission']=data.groupby(['Type of Admission'])['Admission_Deposit'].transform('min')
data['min_Admission_Deposit_per_bed']=data.groupby(['Bed Grade'])['Admission_Deposit'].transform('min')
data['min_Admission_Deposit_per_department']=data.groupby(['Department'])['Admission_Deposit'].transform('min')
data['min_Admission_Deposit_per_hosp']=data.groupby(['Hospital_code'])['Admission_Deposit'].transform('min')

data['mean_Admission_Deposit_per_patient_hosp']=data.groupby(['patientid','Hospital_code'])['Admission_Deposit'].transform('mean')
data['sum_Admission_Deposit_per_patient_hosp']=data.groupby(['patientid','Hospital_code'])['Admission_Deposit'].transform('sum')
data['max_Admission_Deposit_per_patient_hosp']=data.groupby(['patientid','Hospital_code'])['Admission_Deposit'].transform('max')
data['min_Admission_Deposit_per_patient_hosp']=data.groupby(['patientid','Hospital_code'])['Admission_Deposit'].transform('min')

data['illness_count']=data.groupby('Hospital_code')['Severity of Illness'].transform('count')
data['bed_count']=data.groupby('Hospital_code')['Bed Grade'].transform('count')
data['room_count']=data.groupby('Hospital_code')['Available Extra Rooms in Hospital'].transform('count')
data['Department_count']=data.groupby('Hospital_code')['Department'].transform('count')
data['ward_count']=data.groupby('Hospital_code')['Ward_Type'].transform('count')
data['TOA_count']=data.groupby('Hospital_code')['Type of Admission'].transform('count')

data['meanVisitors with Patient_per_patient']=data.groupby(['patientid'])['Visitors with Patient'].transform('mean')
data['meanVisitors with Patient_per_patient']=data.groupby(['patientid'])['Visitors with Patient'].transform('sum')
data['meanVisitors with Patient_per_patient']=data.groupby(['patientid'])['Visitors with Patient'].transform('max')
data['meanVisitors with Patient_per_patient']=data.groupby(['patientid'])['Visitors with Patient'].transform('min')

data['unique_hospital_visited']=data.groupby('patientid')['Hospital_code'].transform('nunique')
data['unique_hospitaltype_visited']=data.groupby('patientid')['Hospital_type_code'].transform('nunique')
data['count_visited_same_hospital']=data.groupby(['patientid','Hospital_code'])['Hospital_code'].transform('count')

data['mean_Admission_Deposit_per_patient']=data.groupby(['patientid'])['Admission_Deposit'].transform('mean')
data['sum_Admission_Deposit_per_patient']=data.groupby(['patientid'])['Admission_Deposit'].transform('sum')
data['max_Admission_Deposit_per_patient']=data.groupby(['patientid'])['Admission_Deposit'].transform('max')
data['min_Admission_Deposit_per_patient']=data.groupby(['patientid'])['Admission_Deposit'].transform('min')

data['mean_visitors_per_patient']=data.groupby(['patientid'])['Visitors with Patient'].transform('mean')
data['unique city']=data.groupby('patientid')['City_Code_Hospital'].transform('nunique')
data['unique illness type']=data.groupby('patientid')['Severity of Illness'].transform('nunique')
data['unique admission type']=data.groupby('patientid')['Type of Admission'].transform('nunique')

data['mean_Admission_Deposit_per_patient_in_same_hospital']=data.groupby(['patientid','Hospital_code'])['Admission_Deposit'].transform('mean')

data['total_no_of_patients_in_hospital']=data.groupby('Hospital_code')['patientid'].transform('count')
data['total_no_of_patients_in_hospital_from_same_city']=data.groupby(['Hospital_code','City_Code_Patient'])['patientid'].transform('count')
data['total_no_of_patients_in_patientcity']=data.groupby('City_Code_Patient')['patientid'].transform('count')
data['total_no_of_patients_in_hospitalcity']=data.groupby('City_Code_Hospital')['patientid'].transform('count')

data['number_of_time_patient_visited']=data.groupby('patientid')['patientid'].transform('count')
data['Total_Hospitals_in_city']=data.groupby('City_Code_Hospital')['Hospital_code'].transform('nunique')
data['Total_Hospitals_in_city_of_same_type']=data.groupby(['City_Code_Hospital','Hospital_type_code'])['Hospital_code'].transform('nunique')

data['total_no_of_patients_per_Department']=data.groupby('Department')['patientid'].transform('count')
data['total_no_of_patients_per_Severity of Illness']=data.groupby('Severity of Illness')['patientid'].transform('count')
data['Total_Hospitals_in_region']=data.groupby('Hospital_region_code')['patientid'].transform('count')
data['total_no_of_patients_in_hospitalward']=data.groupby(['Hospital_code','Ward_Type'])['patientid'].transform('count')
data['total_no_of_patients_in_hospitaldepartment']=data.groupby(['Hospital_code','Department'])['patientid'].transform('count')
data['total_no_of_patients_in_hospitalBedGrade']=data.groupby(['Hospital_code','Bed Grade'])['patientid'].transform('count')
data['total_no_of_patients_in_hospitalSeverityofIllness']=data.groupby(['Hospital_code','Type of Admission'])['patientid'].transform('count')
data['unique_patientcity_in_hospital']=data.groupby('Hospital_code')['City_Code_Patient'].transform('nunique')
data['Average_deposit']=data.groupby('Hospital_code')['Available Extra Rooms in Hospital'].transform('mean')
data['Total_Hospitals_in_patientcity']=data.groupby('City_Code_Patient')['Hospital_code'].transform('nunique')
data['Total_Hospitals_in_region']=data.groupby('Hospital_region_code')['Hospital_code'].transform('nunique')

In [23]:
data["RANK"] = data.groupby("patientid")['patientid'].rank(method="first", ascending=True)
data["RANK_avg"] = data.groupby("patientid")['patientid'].rank(method="average", ascending=True)
data["RANK_max"] = data.groupby("patientid")['patientid'].rank(method="max", ascending=True)
data["RANK_min"] = data.groupby("patientid")['patientid'].rank(method="min", ascending=True)
data["RANK_DIFF"] = data['RANK_max'] - data['RANK_min']
data["RANK_Type_Admission"] = data.groupby(['patientid','Type of Admission'])['patientid'].rank(method='first',ascending=True)
data["RANK_Type_City_Code"] = data.groupby(['patientid','City_Code_Hospital'])['patientid'].rank(method='first',ascending=True)
data["RANK_Type_hosp_Code"] = data.groupby(['patientid','Hospital_code'])['patientid'].rank(method='first',ascending=True)
data["RANK_Type_hosp_type_Code"] = data.groupby(['patientid','Hospital_type_code'])['patientid'].rank(method='first',ascending=True)
data["RANK_Type_rooms"] = data.groupby(['patientid','Available Extra Rooms in Hospital'])['patientid'].rank(method='first',ascending=True)
data["RANK_Type_Department"] = data.groupby(['patientid','Department'])['patientid'].rank(method='first',ascending=True)
data["RANK_Type_Ward_Type"] = data.groupby(['patientid','Ward_Type'])['patientid'].rank(method='first',ascending=True)
data["RANK_Type_Ward_Facility_Code"] = data.groupby(['patientid','Ward_Facility_Code'])['patientid'].rank(method='first',ascending=True)
data["RANK_Type_Hospital_region_code"] = data.groupby(['patientid','Hospital_region_code'])['patientid'].rank(method='first',ascending=True)
data['Rank_City_Code_Patient'] = data.groupby(['patientid','City_Code_Patient'])['patientid'].rank(method='first',ascending=True)
data['Rank_City_Code_Vist'] = data.groupby(['patientid','Visitors with Patient'])['patientid'].rank(method='first',ascending=True)
data['Rank_ill+admiss'] = data.groupby(['patientid','Type of Admission','Severity of Illness'])['patientid'].rank(method='first',ascending=True)
data['Rank_ill+admiss_max'] = data.groupby(['patientid','Type of Admission','Severity of Illness'])['patientid'].rank(method='max',ascending=True)
data['Rank_ill+admiss_min'] = data.groupby(['patientid','Type of Admission','Severity of Illness'])['patientid'].rank(method='min',ascending=True)
data['Rank_ill+admiss_avg'] = data.groupby(['patientid','Type of Admission','Severity of Illness'])['patientid'].rank(method='average',ascending=True)
data['Rank_ill+admiss_diff'] = data['Rank_ill+admiss_max'] - data['Rank_ill+admiss_min']
cols_rank = ['Type of Admission','City_Code_Hospital','Hospital_code','Hospital_type_code','Available Extra Rooms in Hospital','Department','Ward_Type','Ward_Facility_Code',
             'Hospital_region_code','City_Code_Patient','Visitors with Patient']
for i in cols_rank:
    data[f"RANK_{i}_min"] = data.groupby(['patientid',i])['patientid'].rank(method='min',ascending=True)
    data[f"RANK_{i}_max"] = data.groupby(['patientid',i])['patientid'].rank(method='max',ascending=True)
    data[f"RANK_{i}_avg"] = data.groupby(['patientid',i])['patientid'].rank(method='average',ascending=True)
    data[f'diff_{i}_max_min'] = data[f"RANK_{i}_max"] -  data[f"RANK_{i}_min"]

In [24]:
train = data.loc[data.Flag.isin([0])]
test = data.loc[data.Flag.isin([1])]
train.drop(columns={'Flag'},axis=1,inplace=True)
test.drop(columns={'Flag'},axis=1,inplace=True)

In [25]:
x = train.drop(columns={'case_id'},axis=1)
test = test.drop(columns={'case_id'},axis=1)

In [26]:
test.shape

In [27]:
%%time
err = [] 
y_pred_tot_lgm = np.zeros((len(test), 11))
fold = StratifiedKFold(n_splits = 10, shuffle=True).split(x, Y)
i = 1

for train_index, test_index in fold:
    x_train, x_val = x.iloc[train_index], x.iloc[test_index]
    y_train, y_val = Y.iloc[train_index], Y.iloc[test_index]
    KAT = CatBoostClassifier(
        n_estimators = 10000,
        eval_metric = 'Accuracy',
        learning_rate = 0.09,
        depth = 8,
        bagging_temperature=0.3,
        task_type='GPU'
        )
    
    KAT.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=100, verbose=50)
    pred_y = KAT.predict(x_val)
    print(i, " err_lgm: ", accuracy_score(y_val,pred_y))
    err.append(accuracy_score(y_val,pred_y))
    i = i + 1
    y_pred_tot_lgm += KAT.predict_proba(test)
    
sum(err)/10
y_pred_tot_lgm=y_pred_tot_lgm/10

In [32]:
sub=pd.read_csv('../input/av-healthcare-analytics-ii/healthcare/sample_sub.csv')
sub['Stay'] = [np.argmax(x) for x in y_pred_tot_lgm]
sub['Stay']=la.inverse_transform(sub['Stay'])
sub['Stay'].value_counts(normalize=True)
sub.to_csv('final_data.csv', index=False)
